<a href="https://colab.research.google.com/github/Aggraj/Deep-Learning-CS-6910/blob/main/CNN_scratch_5layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Apr 22 10:12:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# set SIZE to "TINY", "MEDIUM", or "LARGE"
# to select one of these three datasets
# TINY dataset: 100 images, 30MB
# MEDIUM dataset: 1000 images, 312MB
# LARGE datast: 12,000 images, 3.6GB

SIZE = "LARGE"

In [ ]:
if SIZE == "TINY":
  src_url = "https://storage.googleapis.com/wandb_datasets/nature_100.zip"
  src_zip = "nature_100.zip"
  DATA_SRC = "nature_100"
  IMAGES_PER_LABEL = 10
  BALANCED_SPLITS = {"train" : 8, "val" : 1, "test": 1}
elif SIZE == "MEDIUM":
  src_url = "https://storage.googleapis.com/wandb_datasets/nature_1K.zip"
  src_zip = "nature_1K.zip"
  DATA_SRC = "nature_1K"
  IMAGES_PER_LABEL = 100
  BALANCED_SPLITS = {"train" : 80, "val" : 10, "test": 10}
elif SIZE == "LARGE":
  src_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
  src_zip = "nature_12K.zip"
  DATA_SRC = "inaturalist_12K/train" # (technically a subset of only 10K images)
  IMAGES_PER_LABEL = 1000
  BALANCED_SPLITS = {"train" : 800, "val" : 100, "test": 100}

In [ ]:
%%capture
!curl -SL $src_url > $src_zip
!unzip $src_zip

# Step 0: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library
*   **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [ ]:
!pip install wandb -qq
import wandb
wandb.login()

     |████████████████████████████████| 2.1MB 15.5MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 163kB 72.0MB/s 
     |████████████████████████████████| 133kB 59.3MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import os
from random import shuffle

# source directory for all raw data
SRC = DATA_SRC
# number of images per class label
# the total number of images is 10X this (10 classes)
TOTAL_IMAGES = IMAGES_PER_LABEL * 10
PROJECT_NAME = "CNN_scratch"
PREFIX = "inat" # convenient for tracking local data

# Step 1: Upload raw data

In [ ]:
RAW_DATA_AT = "_".join([PREFIX, "raw_data", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")

# SRC_DIR contains 10 folders, one for each of 10 class labels
# each folder contains images of the corresponding class
labels = os.listdir(SRC)
for l in labels:
  imgs_per_label = os.path.join(SRC, l)
  if os.path.isdir(imgs_per_label):
    imgs = os.listdir(imgs_per_label)
    # randomize the order
    shuffle(imgs)
    img_file_ids = imgs[:IMAGES_PER_LABEL]
    for f in img_file_ids:
      file_path = os.path.join(SRC, l, f)
      # add file to artifact by full path
      raw_data_at.add_file(file_path, name=l + "/" + f)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

wandb: Currently logged in as: chaxin (use `wandb login --relogin` to force relogin)


# Step 2: Prepare a data split


In [ ]:
run = wandb.init(project=PROJECT_NAME, job_type="data_split")

# find the most recent ("latest") version of the full raw data
# you can of course pass around programmatic aliases and not string literals
data_at = run.use_artifact(RAW_DATA_AT + ":latest")
# download it locally (for illustration purposes/across hardware; you can
# also sync/version artifacts by reference)
data_dir = data_at.download()

# create balanced train, val, test splits
# each count is the number of images per label
DATA_SPLITS = BALANCED_SPLITS

ats = {}
# wrap artifacts in dictionary for convenience
for split, count in DATA_SPLITS.items():
  ats[split] = wandb.Artifact("_".join([PREFIX, split, "data", str(count*10)]), 
                              "_".join([split, "data"]))

labels = os.listdir(data_dir)
for l in labels:
  if l.startswith("."): # skip non-label file
    continue
  imgs_per_label = os.listdir(os.path.join(data_dir, l))
  shuffle(imgs_per_label)
  start_id = 0
  for split, count in DATA_SPLITS.items():
    # take a subset
    split_imgs = imgs_per_label[start_id:start_id+count]
    for img_file in split_imgs:
      full_path = os.path.join(data_dir, l, img_file)
      # add file to artifact by full path
      # note: pass the label to the name parameter to retain it in
      # the data structure 
      ats[split].add_file(full_path, name = os.path.join(l, img_file))
    start_id += count

# save all three artifacts to W&B
# note: yes, in this example, we are cheating and have labels for the "test" data ;)
for split, artifact in ats.items():
  run.log_artifact(artifact)

run.finish()

wandb: Downloading large artifact inat_raw_data_10000:latest, 3044.53MB. 10000 files... Done. 0:0:0


In [ ]:
# EXPERIMENT CONFIG
#---------------------------
# if you modify these, make sure the total count is less than or equal to
# the number of files uploaded for that split in the train/val data artifact
NUM_TRAIN = BALANCED_SPLITS["train"] * 10
NUM_VAL = BALANCED_SPLITS["val"] * 10
NUM_EPOCHS = 1 # set low for demo purposes; try 3, 5, or as many as you like

# model name
# if you want to train a sufficiently different model, give this a new name
# to start a new lineage for the model, instead of just incrementing the
# version of the old model
MODEL_NAME = "iv3_trained"

# folder in which to save initial, untrained model
INIT_MODEL_DIR = "init_model_keras_iv3"

# folder in which to save the final, trained model
# if you want to train a sufficiently different model, give this a new name
# to start a new lineage for the model, instead of just incrementing the
# version of the old model
FINAL_MODEL_DIR = "trained_keras_model_iv3"

import numpy as np
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from wandb.keras import WandbCallback

# experiment configuration saved to W&B
config_defaults = {
  "num_train" : NUM_TRAIN,
  "num_val" : NUM_VAL,
  "epochs" : NUM_EPOCHS,
  "num_classes" : 10,
  "num_neurons" : 256,
  # inceptionV3 settings
  "img_width" : 299,
  "img_height": 299,
  "batch_size" : 50,
  'filter': [64,64,32,32,16],
  'filter_size' : [9,8,7,6,5],
  'drop':0.5,
  'agumentation':1
}


def CNN_model(filter,filter_size,num_neurons,epochs,drop):


  model = Sequential()

    # # model.add(Conv2D(args['num_filter'][0], args['num_filter'][0], activation=args['activation'], input_shape=(4,7,1)))
  model.add(Conv2D(filter[0],(filter_size[0],filter_size[0]), activation='relu', input_shape=(299,299,3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Conv2D(filter[1],(filter_size[1],filter_size[1]), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Conv2D(filter[2],(filter_size[2],filter_size[2]), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Conv2D(filter[3],(filter_size[3],filter_size[3]), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Conv2D(filter[4],(filter_size[4],filter_size[4]), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(num_neurons, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  model.summary()
  return model

    # model.fit(xTrain, yTrain, batch_size=128,epochs=num_epoch, validation_data=(xval, yval),verbose=1)

def train():
  """ Main training loop. This is called pretrain because it freezes
  the InceptionV3 layers of the model and only trains the new top layers
  on the new data.   subsequent training phase would unfreeze all the layers
  and finetune the whole model on the new data""" 
  # track this experiment with wandb: all runs will be sent
  # to the given project name
  run = wandb.init(project=PROJECT_NAME, job_type="train", config=config_defaults)
  cfg = wandb.config

  # artifact names
  train_at = os.path.join(PROJECT_NAME, PREFIX + "_train_data_" + str(NUM_TRAIN)) + ":latest"
  val_at = os.path.join(PROJECT_NAME, PREFIX + "_val_data_" + str(NUM_VAL)) + ":latest"

  train_data = run.use_artifact(train_at, type='train_data')
  train_dir = train_data.download()
  val_data = run.use_artifact(val_at, type='val_data')
  val_dir = val_data.download()

  # create train and validation data generators
  if cfg.agumentation == 1 :
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range = 36,
        shear_range=0.3,
        zoom_range=0.4,
        horizontal_flip=True,
        fill_mode = 'nearest')
  else :
    train_datagen = ImageDataGenerator(rescale=1. / 255)
  
  val_datagen = ImageDataGenerator(rescale=1. / 255)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(cfg.img_width, cfg.img_height),
    batch_size=cfg.batch_size,
    class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(cfg.img_width, cfg.img_height),
    batch_size=cfg.batch_size,
    class_mode='categorical')

  # instantiate model and callbacks
  model = CNN_model(cfg.filter,cfg.filter_size,cfg.num_neurons,cfg.epochs,cfg.drop)

  # log initial model before training
  model_artifact = wandb.Artifact(
            "CNN_1", type="model",
            description="CNN_m1",
            metadata=dict(cfg))

  model.save(INIT_MODEL_DIR)
  model_artifact.add_dir(INIT_MODEL_DIR)
  run.log_artifact(model_artifact)
  callbacks = [WandbCallback()]

  # train!
  model.fit(
    train_generator,
    steps_per_epoch = cfg.num_train // cfg.batch_size,
    epochs=cfg.epochs,
    validation_data=val_generator,
    callbacks = callbacks,
    validation_steps=cfg.num_val // cfg.batch_size)

  # save trained model as artifact
  trained_model_artifact = wandb.Artifact(
            'CNN11', type="model",
            description="trained CNN ",
            metadata=dict(cfg))

  model.save(FINAL_MODEL_DIR)
  trained_model_artifact.add_dir(FINAL_MODEL_DIR)
  run.log_artifact(trained_model_artifact)
  run.finish()

In [ ]:
train()


In [ ]:
%pip install wandb -q
import wandb
wandb.login()

True

# Step 4: Load model for inference


In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'filter': {
            'values': [[32,32,64,128,256],[64,128,128,256,256],[32,32,32,32,32],[64,64,64,64,64]]
        },
        'filter_size': {
            'values': [[5,5,3,3,2], [5,3,3,2,2],[7,5,3,3,2]]
        },
        'drop': {
            'values': [0.3,0.5]
        },
        'num_neurons': {
            'values': [200,500]
        },
        'epochs':{
            'values' :[10,15]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="chaxin", project="Assignment 1")

Create sweep with ID: p7sy2gs6
Sweep URL: https://wandb.ai/chaxin/Assignment%201/sweeps/p7sy2gs6


In [ ]:
sweep_id = 'l9v4q0no'

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 1j2gu4li with config:
wandb: 	drop: 0.5
wandb: 	epochs: 10
wandb: 	filter: [32, 32, 64, 128, 256]
wandb: 	filter_size: [7, 5, 3, 3, 2]
wandb: 	num_neurons: 200
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: Downloading large artifact inat_train_data_8000:latest, 2443.86MB. 8000 files... Done. 0:0:0
wandb: Downloading large artifact inat_val_data_1000:latest, 300.06MB. 1000 files... Done. 0:0:0


Found 7999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 293, 293, 32)      4736      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 146, 146, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 146, 146, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 146, 146, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 142, 142, 32)      25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 71, 71, 32)        0         
____________________________________

wandb: Adding directory to artifact (./init_model_keras_iv3)... Done. 0.2s


Epoch 1/10
160/160 [==============================] - 243s 1s/step - loss: 3.1894 - accuracy: 0.1174 - val_loss: 2.5665 - val_accuracy: 0.0940
Epoch 2/10
160/160 [==============================] - 221s 1s/step - loss: 2.2159 - accuracy: 0.1832 - val_loss: 2.8784 - val_accuracy: 0.1000
Epoch 3/10
160/160 [==============================] - 222s 1s/step - loss: 2.1209 - accuracy: 0.2284 - val_loss: 3.4990 - val_accuracy: 0.1000
Epoch 4/10
160/160 [==============================] - 221s 1s/step - loss: 2.0723 - accuracy: 0.2460 - val_loss: 2.7835 - val_accuracy: 0.1110
Epoch 5/10
160/160 [==============================] - 221s 1s/step - loss: 2.0512 - accuracy: 0.2577 - val_loss: 3.1377 - val_accuracy: 0.1350
Epoch 6/10
160/160 [==============================] - 221s 1s/step - loss: 2.0047 - accuracy: 0.2800 - val_loss: 3.0400 - val_accuracy: 0.1900
Epoch 7/10
160/160 [==============================] - 221s 1s/step - loss: 2.0204 - accuracy: 0.2774 - val_loss: 2.8963 - val_accuracy: 0.1040

...

wandb: Downloading large artifact inat_train_data_8000:latest, 2435.89MB. 8000 files... Done. 0:0:0
wandb: Downloading large artifact inat_val_data_1000:latest, 299.80MB. 1000 files... Done. 0:0:0


Found 8000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 295, 295, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 147, 147, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 147, 147, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 147, 147, 64)      256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 143, 143, 64)      102464    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 71, 71, 64)        0         
_____________________________________

wandb: Adding directory to artifact (./init_model_keras_iv3)... Done. 0.2s


Epoch 1/15
160/160 [==============================] - 225s 1s/step - loss: 2.9512 - accuracy: 0.1356 - val_loss: 2.8998 - val_accuracy: 0.0971
Epoch 2/15
160/160 [==============================] - 224s 1s/step - loss: 2.4013 - accuracy: 0.1898 - val_loss: 2.7986 - val_accuracy: 0.1141
Epoch 3/15
160/160 [==============================] - 227s 1s/step - loss: 2.1482 - accuracy: 0.2224 - val_loss: 2.9253 - val_accuracy: 0.1181
Epoch 4/15
160/160 [==============================] - 221s 1s/step - loss: 2.0595 - accuracy: 0.2600 - val_loss: 2.8940 - val_accuracy: 0.1011
Epoch 5/15
160/160 [==============================] - 222s 1s/step - loss: 2.0345 - accuracy: 0.2729 - val_loss: 2.8244 - val_accuracy: 0.1081
Epoch 6/15
160/160 [==============================] - 222s 1s/step - loss: 2.0102 - accuracy: 0.2830 - val_loss: 2.9927 - val_accuracy: 0.1071
Epoch 7/15
160/160 [==============================] - 222s 1s/step - loss: 1.9730 - accuracy: 0.3005 - val_loss: 2.7160 - val_accuracy: 0.1441

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
import os
run = wandb.init(project=PROJECT_NAME, job_type="inference")
# use the latest version of the model
model_at = run.use_artifact(MODEL_NAME + ":latest")
# download the directory in which the model is saved
model_dir= model_at.download()
print("model: ", model_dir)
model = keras.models.load_model(model_dir)

TEST_DATA_AT = PREFIX + "_test_data_" + str(BALANCED_SPLITS["test"]*10) + ":latest"
test_data_at = run.use_artifact(TEST_DATA_AT)
test_dir = test_data_at.download()

imgs = []
class_labels = os.listdir(test_dir)
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join(test_dir, l))
  for img in imgs_per_class:
    img_path = os.path.join(test_dir, l, img)
    img = image.load_img(img_path, target_size=(299, 299))
    img = image.img_to_array(img)
    # don't forget to rescale test images to match the range of inputs
    # to the network
    img = np.expand_dims(img/255.0, axis=0)
    imgs.append(img)

preds = {}
imgs = np.vstack(imgs)
classes = model.predict(imgs, batch_size=32)
for c in classes:
  class_id = np.argmax(c)
  if class_id in preds:
    preds[class_id] += 1
  else:
    preds[class_id] = 1

# print the counts of predicted labels as a quick sanity check
# note that for tiny/medium datasets, this won't be very meaningful
print(preds)
run.finish()


wandb: Downloading large artifact iv3_trained:latest, 233.43MB. 3 files... Done. 0:0:0


model:  ./artifacts/iv3_trained:v2
{5: 11, 0: 17, 7: 20, 4: 6, 8: 8, 2: 6, 3: 12, 6: 8, 9: 6, 1: 6}


# More about Weights & Biases
We're always free for academics and open source projects. Email carey@wandb.com with any questions or feature suggestions. Here are some more resources:

1. [Documentation](http://docs.wandb.com) - Python docs
2. [Gallery](https://app.wandb.ai/gallery) - example reports in W&B
3. [Articles](https://www.wandb.com/articles) - blog posts and tutorials
4. [Community](wandb.me/slack) - join our Slack community forum